# Анализ базы данных книжного сервиса

**Цель исследования** заключается в анализе базы данных книжного сервиса по подписке. Главная задача - выявить ключевые метрики и характеристики книг и пользователей, чтобы сформулировать ценностное предложение для нового продукта. Конкретные цели включают подсчет количества книг после 1 января 2000 года, определение количества обзоров и средней оценки для каждой книги, выявление наиболее активных издательств и популярных авторов, а также анализ среднего количества обзоров от активных пользователей. 

### Доступ к базе данных и исследование таблиц

Установим доступ к базе данных, загрузим таблицы и выведем первые строк каждой из них:

In [5]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# список таблиц для выполнения запросов
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

# выполняем SQL-запросы и выводим первые 5 строк каждой таблицы
for table in tables:
    query = f'SELECT * FROM {table} LIMIT 5'
    con = engine.connect()
    df = pd.io.sql.read_sql(sql=text(query), con=con)
    print(f'Первые 5 строк таблицы {table}:')
    display(df)
    print('\n')

Первые 5 строк таблицы books:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268




Первые 5 строк таблицы authors:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd




Первые 5 строк таблицы publishers:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company




Первые 5 строк таблицы ratings:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2




Первые 5 строк таблицы reviews:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Задания

#### Посчитаем, сколько книг вышло после 1 января 2000 года:

In [23]:
query = '''
SELECT COUNT(book_id) AS total_books
FROM books
WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(sql=text(query), con=con)

,total_books
0,819


С 1 января 2000 года вышло 819 книг.

#### Для каждой книги посчитаем количество обзоров и среднюю оценку:

In [63]:
query = '''
SELECT
    b.title,
    COUNT(DISTINCT r.review_id) AS total_reviews,
    AVG(ra.rating) AS average_rating
FROM
    books b
LEFT JOIN
    reviews r ON b.book_id = r.book_id
LEFT JOIN
    ratings ra ON b.book_id = ra.book_id
GROUP BY
    b.book_id
ORDER BY
    total_reviews DESC, average_rating DESC
'''

pd.io.sql.read_sql(sql=text(query), con=con)

,title,total_reviews,average_rating
0,Twilight (Twilight #1),7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Book Thief,6,4.264151
4,The Glass Castle,6,4.206897
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,Essential Tales and Poems,0,4.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


Больше всего обзоров на следующие книги: Memoirs of a Geisha (8 обзоров), Twilight (Twilight #1) (7 обзоров)

Также стоит отметить, что в десятке — 2 книги из серии романов про Гарри Поттера.

#### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц:

In [26]:
query = '''
SELECT 
    p.publisher,
    COUNT(b.book_id) AS book_count
FROM 
    books b
JOIN 
    publishers p ON b.publisher_id = p.publisher_id
WHERE 
    b.num_pages > 50
GROUP BY 
    p.publisher
ORDER BY 
    book_count DESC
LIMIT 1;
'''

pd.io.sql.read_sql(sql=text(query), con=con)

,publisher,book_count
0,Penguin Books,42


Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц — 42.

#### Определим автора с самой высокой средней оценкой книг:

In [27]:
query = '''
SELECT
    a.author,
    AVG(r2.rating) AS average_rating
FROM
    authors a
JOIN
    books b ON a.author_id = b.author_id
JOIN
    (
        SELECT
            book_id,
            AVG(rating) AS rating_avg
        FROM
            ratings
        GROUP BY
            book_id
        HAVING
            COUNT(rating) >= 50
    ) r ON b.book_id = r.book_id
JOIN
    ratings r2 ON b.book_id = r2.book_id
GROUP BY
    a.author
ORDER BY
    average_rating DESC
LIMIT 1;

'''

pd.io.sql.read_sql(sql=text(query), con=con)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Автором с самой высокой средней оценкой книг являет Дж. К. Роулинг — средний рейтинг её книг составляет 4.29.

#### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок:

In [28]:
query = '''
SELECT
    AVG(review_count) AS average_review_count
FROM
    (
        SELECT
            username,
            COUNT(review_id) AS review_count
        FROM
            reviews
        WHERE
            username IN (
                SELECT
                    username
                FROM
                    ratings
                GROUP BY
                    username
                HAVING
                    COUNT(rating_id) > 48
            )
        GROUP BY
            username
    ) AS subquery
'''

pd.io.sql.read_sql(sql=text(query), con=con)

,average_review_count
0,24.0


Среднее количество обзоров от таких пользователей — 24.

#### Выведите таблицу, которая будет содержать по году публикации:
количество издательств, \
выпущенных книг и \
сколько всего тысяч страниц было в изданных книгах \
(отобразить только те года, в которых издано более 30 книг)

In [22]:
query = '''
SELECT
    EXTRACT(YEAR FROM publication_date) AS publication_year,
    COUNT(DISTINCT publisher_id) AS count_publishers,
    COUNT(book_id) AS count_books,
    SUM(num_pages) / 1000.0 AS total_pages_thousands
FROM
    books
WHERE
    EXTRACT(YEAR FROM publication_date) IN (
        SELECT EXTRACT(YEAR FROM publication_date)
        FROM books
        GROUP BY EXTRACT(YEAR FROM publication_date)
        HAVING COUNT(book_id) > 30
    )
GROUP BY
    publication_year
ORDER BY
    publication_year;
'''

pd.io.sql.read_sql(sql=text(query), con=con)

,publication_year,count_publishers,count_books,total_pages_thousands
0,1999.0,26,41,15.763
1,2000.0,35,38,13.328
2,2001.0,41,60,21.758
3,2002.0,62,94,38.597
4,2003.0,65,105,41.423
5,2004.0,88,124,46.779
6,2005.0,89,139,55.967
7,2006.0,109,184,68.302
8,2007.0,38,50,18.258


#### Выведите в одной таблице два числа — среднюю оценку тех книг, на которые написало отзывов более 3 человек и отдельно среднюю оценку остальных книг:

In [60]:
query = '''
SELECT 
    (SELECT AVG(rating)
     FROM ratings
     WHERE book_id IN (SELECT book_id
                       FROM reviews
                       GROUP BY book_id
                       HAVING COUNT(DISTINCT review_id) > 3)) AS avg_rating_4plus_reviews,
    
    (SELECT AVG(rating)
     FROM ratings
     WHERE book_id NOT IN (SELECT book_id
                           FROM reviews
                           GROUP BY book_id
                           HAVING COUNT(review_id) > 3)) AS avg_rating_3orless_reviews;
'''

pd.io.sql.read_sql(sql=text(query), con=con)

,avg_rating_4plus_reviews,avg_rating_3orless_reviews
0,3.949505,3.890375


Таким образом, рейтинг книг, у которых больше 3 отзывов, формально выше рейтинга остальных книг. Но, в целом, отличие является незначительным и можно сказать что они примерно равны.

### Общий вывод

В результате анализа базы данных книжного сервиса по подписке мы получили ценные метрики и характеристики книг и пользователей. Мы выяснили, что с 1 января 2000 года было выпущено 819 книг, и нашли топ-3 книг с наибольшим количеством обзоров. Выяснили, что серия книг о Гарри Поттере занимает значимое место среди самых обсуждаемых произведений.

Также мы выявили наиболее продуктивное издательство — Penguin Books, которое выпустило 42 книги толще 50 страниц, что может быть важной информацией для формулирования стратегии сотрудничества с издательствами.

Одним из ключевых авторов с самой высокой средней оценкой книг является Дж. К. Роулинг, чьи произведения имеют средний рейтинг 4.29, что может привлечь большее количество пользователей, предпочитающих книги с высокими оценками.

Наконец, мы оценили активность пользователей и выяснили, что те, кто ставит более 48 оценок, также проявляют активность в написании обзоров — в среднем они пишут 24 ревью.

Выводы из этого анализа помогут сформулировать ценностное предложение для нового продукта, привлечь больше пользователей и повысить их удовлетворенность сервисом.